# Creation table contenant le test de la liste numéro de voie

# Objective(s)

*  Comparer les listes INSEE/INPI contenant les digits de l’adresse:
  * `intersection_numero_voie`: Nombre de numéros en commun
  * `union_numero_voie`: Ensemble des numéros des deux listes

# Metadata

* Epic: Epic 6
* US: US 5
* Date Begin: 9/29/2020
* Duration Task: 0
* Description: Creation de la table  ets_insee_inpi_list_num_voie qui compare la liste des digits de l’adresse INPI avec la liste de l’adresse INSEE
* Step type: Transform table
* Status: Active
  * Change Status task: Active
  * Update table: Modify rows
* Source URL:  
* Task type: Jupyter Notebook
* Users: Thomas Pernet
* Watchers: Thomas Pernet
* User Account: https://937882855452.signin.aws.amazon.com/console
* Estimated Log points: 5
* Task tag: #athena,#sql,#data-preparation,#inpi,#insee
* Toggl Tag: #documentation

# Input Cloud Storage [AWS/GCP]

## Table/file

* Origin: 
* Athena
* Name: 
* ets_insee_inpi
* Github: 
  * https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/11_sumup_siretisation/00_merge_ets_insee_inpi.md

# Destination Output/Delivery

## Table/file

* Origin: 
* Athena
* Name:
* ets_insee_inpi_list_num_voie
* GitHub:
* https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/11_sumup_siretisation/02_test_list_num_voie.md


## Connexion serveur

In [ ]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import os, shutil

path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = 'eu-west-3')

region = 'eu-west-3'
bucket = 'calfdata'

In [ ]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False) 

In [ ]:
pandas_setting = True
if pandas_setting:
    cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Introduction

Pour préparer le test sur la voie, il faut avoir deux variables indiquant la ressemblance de la liste des numéros de l'INSEE et celle de l'INPI. Ainsi, dans ce notebook, nous allons créer:

* `intersection_numero_voie`: Nombre de numéros en commun
* `union_numero_voie`: Ensemble des numéros des deux listes

Le tableau ci dessous récapitule les sous-ensembles 1 à 6, qui vont de la création des types de ressemble entre l'adresse de l'INSEE et de l'INPI jusqu'aux tests sur la date, l'état administratif et de l'ensemble

 | Rang | Nom_variable                              | Dependence                                    | Notebook                           | Difficulte | Table_generee                                                                                                                                                            | Variables_crees_US                                                                 | Possibilites                  |
|------|-------------------------------------------|-----------------------------------------------|------------------------------------|------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------|-------------------------------|
| 1    | status_cas                                |                                               | 01_cas_de_figure                   | Moyen      | ets_insee_inpi_status_cas                                                                                                                                              | status_cas,intersection,pct_intersection,union_,inpi_except,insee_except           | CAS_1,CAS_2,CAS_3,CAS_4,CAS_5 |
| 2    | test_list_num_voie                        | intersection_numero_voie,union_numero_voie    | 03_test_list_num_voie              | Moyen      | ets_insee_inpi_list_num_voie                                                                                                                                           | intersection_numero_voie,union_numero_voie                                         | FALSE,NULL,TRUE,PARTIAL       |
| 3    | test_enseigne                             | list_enseigne,enseigne                        | 04_test_enseigne                   | Moyen      | ets_insee_inpi_list_enseigne                                                                                                                                           | list_enseigne_contain                                                              | FALSE,NULL,TRUE               |
| 4    | test_pct_intersection                     | pct_intersection,index_id_max_intersection    | 06_creation_nb_siret_siren_max_pct | Facile     | ets_insee_inpi_var_group_max                                                                                                                                           | count_inpi_index_id_siret,count_inpi_siren_siret,index_id_max_intersection         | FALSE,TRUE                    |
| 4    | test_index_id_duplicate                   | count_inpi_index_id_siret                     | 06_creation_nb_siret_siren_max_pct | Facile     | ets_insee_inpi_var_group_max                                                                                                                                           | count_inpi_index_id_siret,count_inpi_siren_siret,index_id_max_intersection         | FALSE,TRUE                    |
| 4    | test_siren_insee_siren_inpi               | count_initial_insee,count_inpi_siren_siret    | 06_creation_nb_siret_siren_max_pct | Facile     | ets_insee_inpi_var_group_max                                                                                                                                           | count_inpi_index_id_siret,count_inpi_siren_siret,index_id_max_intersection         | FALSE,TRUE                    |
| 5    | test_similarite_exception_words           | max_cosine_distance                           | 08_calcul_cosine_levhenstein       | Difficile  | ets_insee_inpi_similarite_max_word2vec                                                                                                                                 | unzip_inpi,unzip_insee,max_cosine_distance,levenshtein_distance,key_except_to_test | FALSE,NULL,TRUE               |
| 5    | test_distance_levhenstein_exception_words | levenshtein_distance                          | 08_calcul_cosine_levhenstein       | Difficile  | ets_insee_inpi_similarite_max_word2vec                                                                                                                                 | unzip_inpi,unzip_insee,max_cosine_distance,levenshtein_distance,key_except_to_test | FALSE,NULL,TRUE               |
| 6    | test_date                                 | datecreationetablissement,date_debut_activite | 10_match_et_creation_regles.md     | Facile     | ets_insee_inpi_list_num_voie,ets_insee_inpi_list_enseigne,ets_insee_inpi_similarite_max_word2vec,ets_insee_inpi_status_cas,ets_insee_inpi_var_group_max,ets_insee_inpi |                                                                                    | FALSE,TRUE                    |
| 6    | test_siege                                | status_ets,etablissementsiege                 | 10_match_et_creation_regles.md     | Facile     | ets_insee_inpi_list_num_voie,ets_insee_inpi_list_enseigne,ets_insee_inpi_similarite_max_word2vec,ets_insee_inpi_status_cas,ets_insee_inpi_var_group_max,ets_insee_inpi |                                                                                    | FALSE,TRUE,NULL               |
| 6    | test_status_admin                         | etatadministratifetablissement,status_admin   | 10_match_et_creation_regles.md     | Facile     | ets_insee_inpi_list_num_voie,ets_insee_inpi_list_enseigne,ets_insee_inpi_similarite_max_word2vec,ets_insee_inpi_status_cas,ets_insee_inpi_var_group_max,ets_insee_inpi |                                                                                    | FALSE,NULL,TRUE               |


In [ ]:
s3_output = 'SQL_OUTPUT_ATHENA'
database = 'ets_siretisation'

In [ ]:
query = """
DROP TABLE ets_siretisation.ets_insee_inpi_list_num_voie;
"""
s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [ ]:
create_table = """
CREATE TABLE ets_siretisation.ets_insee_inpi_list_num_voie
WITH (
  format='PARQUET'
) AS
SELECT 
row_id,
siret, 
list_numero_voie_matching_inpi, 
list_numero_voie_matching_insee,
CAST(
      cardinality(
        array_distinct(
          array_intersect(
            list_numero_voie_matching_inpi, 
            list_numero_voie_matching_insee
          )
        )
      ) AS DECIMAL(10, 2)
    ) as intersection_numero_voie, 
    CAST(
      cardinality(
        array_distinct(
          array_union(
            list_numero_voie_matching_inpi, 
            list_numero_voie_matching_insee
          )
        )
      ) AS DECIMAL(10, 2)
    ) as union_numero_voie
FROM ets_siretisation.ets_insee_inpi 
"""
s3.run_query(
            query=create_table,
            database=database,
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [ ]:
query = """
SELECT *
FROM ets_siretisation.ets_insee_inpi_list_num_voie
LIMIT 10
"""

tb = s3.run_query(
            query=query,
            database=database,
            s3_output=s3_output,
  filename = 'test_list_num_voie', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

# Analyse

1. Vérifier que le nombre de lignes est indentique avant et après la création des variables
2. Calculer le nombre d'occurence pour `list_numero_voie_matching_inpi`, et `list_numero_voie_matching_insee`
    - Afficher le top 50 et bottom 50
3. Calculer le nombre d'occurence par cardinalité pour `list_numero_voie_matching_inpi`, et `list_numero_voie_matching_insee`
4. Donner la distribution de  `intersection_numero_voie` et `union_numero_voie`
    - distribution -> 0.1,0.25,0.5,0.75,0.8,0.95

## 1. Vérifier que le nombre de lignes est indentique avant et après la création des variables

In [ ]:
query = """
SELECT COUNT(*) as CNT
FROM ets_siretisation.ets_insee_inpi
"""
s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'count_ets_insee_inpi', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

In [ ]:
query = """
SELECT COUNT(*) as CNT
FROM ets_siretisation.ets_insee_inpi_list_num_voie

"""
s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'count_ets_insee_inpi_list_num', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

## 2. Calculer le nombre d'occurence pour `list_numero_voie_matching_inpi`, et `list_numero_voie_matching_insee`

`list_numero_voie_matching_inpi`



In [ ]:
query = """
SELECT list_numero_voie_matching_inpi, COUNT(*) as cnt
FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"
GROUP BY list_numero_voie_matching_inpi
ORDER BY cnt DESC
"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'occurence', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

- Top 50

In [ ]:
tb.head(50)

- Bottom 50

In [ ]:
tb.tail(50)

`list_numero_voie_matching_insee`

In [ ]:
query = """
SELECT list_numero_voie_matching_insee, COUNT(*) as cnt
FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"
GROUP BY list_numero_voie_matching_insee
ORDER BY cnt DESC
"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'occurence', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

- Top 50

In [ ]:
tb.head(50)

- Bottom 50

In [ ]:
tb.tail(50)

## 3. Calculer le nombre d'occurence par cardinalité pour `list_numero_voie_matching_inpi`, et `list_numero_voie_matching_insee`

`list_numero_voie_matching_inpi`

In [ ]:
query = """
SELECT  CARDINALITY(list_numero_voie_matching_inpi) as cadinality, COUNT(*) as cnt
FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"
GROUP BY CARDINALITY(list_numero_voie_matching_inpi)
ORDER BY cnt DESC
"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'occurence', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

`list_numero_voie_matching_insee`

In [ ]:
query = """
SELECT  CARDINALITY(list_numero_voie_matching_insee) as cadinality, COUNT(*) as cnt
FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"
GROUP BY CARDINALITY(list_numero_voie_matching_insee)
ORDER BY cnt DESC
"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'occurence', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

## 4. Donner la distribution de  `intersection_numero_voie` et `union_numero_voie`

- distribution -> 0.1,0.25,0.5,0.75,0.8,0.95

`intersection_numero_voie`

In [ ]:
query = """
WITH dataset AS (
  
  SELECT 
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      intersection_numero_voie,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"  
    ) 
    
    SELECT 
    pct, 
    value AS  intersection_numero_voie
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)

"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'distribution', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

`union_numero_voie`

In [ ]:
query = """
WITH dataset AS (
  
  SELECT 
  MAP(
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95],
    approx_percentile(
      union_numero_voie,
    ARRAY[0.1,0.25,0.5,0.75,0.8,0.95])
    ) AS nest
    FROM "ets_siretisation"."ets_insee_inpi_list_num_voie"  
    ) 
    
    SELECT 
    pct, 
    value AS  union_numero_voie
    FROM dataset
    CROSS JOIN UNNEST(nest) as t(pct, value)

"""
tb = s3.run_query(
            query=query,
            database='ets_siretisation',
            s3_output=s3_output,
  filename = 'distribution', ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )
tb

# Generation report

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [ ]:
def create_report(extension = "html", keep_code = False):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    if keep_code:
        os.system('jupyter nbconvert --to {} {}'.format(
    extension,notebookname))
    else:
        os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [ ]:
create_report(extension = "html", keep_code = True)